<a href="https://colab.research.google.com/github/marlon-br/glow-tts-colab/blob/master/glow_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content
!git clone https://github.com/NVIDIA/waveglow waveglow
%cd /content/waveglow
!gdown https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
%ls
%cd /content


/content
fatal: destination path 'waveglow' already exists and is not an empty directory.
/content/waveglow
Downloading...
From: https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
To: /content/waveglow/waveglow_256channels_universal_v5.pt
676MB [00:03, 190MB/s]
config.json*      glow.py       requirements.txt
convert_model.py  inference.py  tacotron2/
denoiser.py       LICENSE       train.py
distributed.py    mel2samp.py   waveglow_256channels_universal_v5.pt
glow_old.py       README.md*    waveglow_logo.png*


In [11]:
%cd /content
!pip3 uninstall apex

!git clone https://www.github.com/nvidia/apex apex
%cd /content/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd /content

/content
Uninstalling apex-0.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/apex-0.1-py3.6.egg
Proceed (y/n)? y
  Successfully uninstalled apex-0.1
fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-21hn8lup
Created temporary directory: /tmp/pip-req-tracker-pogulrrf
Created requirements tracker '/tmp/pip-req-tracker-pogulrrf'
Created temporary directory: /tmp/pip-install-jqy0_xbw
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-4lflpc71
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-pogulrrf'
    Running setup.py (path:/tmp/pip-req-build-4lflpc71/setup.py) egg_info for package from file:///con

In [12]:
import torch

%cd /content/waveglow
waveglow_path = '/content/waveglow/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()
%cd /content

/content/waveglow


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

In [13]:
from apex import amp
waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


In [14]:
%cd /content/
!git clone https://github.com/jaywalnut310/glow-tts glow-tts

/content
Cloning into 'glow-tts'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 75 (delta 22), reused 35 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [66]:
%cd /content/glow-tts
import utils
!ls
!mkdir /content/glow-tts/logs

!cp /content/glow-tts/configs/base.json /content/glow-tts/logs/config.json
%cd /content/glow-tts/logs
!gdown https://drive.google.com/uc?id=1JiCMBVTG4BMREK8cT3MYck1MgYvwASL0
%cd /content

/content/glow-tts
attentions.py	     filelists	      modules.py       stft.py
audio_processing.py  inference.ipynb  monotonic_align  text
commons.py	     init.py	      pretrained.pth   train_ddi.sh
configs		     LICENSE	      __pycache__      train.py
data		     logs	      README.md        utils.py
data_utils.py	     models.py	      resources
mkdir: cannot create directory ‘/content/glow-tts/logs’: File exists
/content/glow-tts/logs
Downloading...
From: https://drive.google.com/uc?id=1JiCMBVTG4BMREK8cT3MYck1MgYvwASL0
To: /content/glow-tts/logs/pretrained.pth
115MB [00:00, 203MB/s]
/content


In [64]:
%cd /content/glow-tts/monotonic_align/
!python setup.py install
from monotonic_align.core import maximum_path_c




/content/glow-tts/monotonic_align
running install
running build
running build_ext
running install_lib
running install_egg_info
Removing /usr/local/lib/python3.6/dist-packages/monotonic_align-0.0.0.egg-info
Writing /usr/local/lib/python3.6/dist-packages/monotonic_align-0.0.0.egg-info


In [57]:
!pip install Unidecode

     |████████████████████████████████| 245kB 3.5MB/s 


In [67]:
%cd /content/glow-tts/logs
%ls

/content/glow-tts/logs
config.json  pretrained.pth


In [84]:
%cd /content/glow-tts

import models
from text import text_to_sequence, cmudict
from text.symbols import symbols
import glob
import os

model_dir = "/logs"
hps = utils.get_hparams_from_dir(model_dir)

model = models.FlowGenerator(
                              len(symbols),
                              out_channels=hps.data.n_mel_channels,
                              **hps.model).to("cuda")

f_list = glob.glob(os.path.join("/logs", "*.pth"))
#f_list.sort(key=lambda f: int("".join(filter(str.isdigit, f))))
x = f_list[-1]
print(x)

#checkpoint_path = utils.latest_checkpoint_path(model_dir)
utils.load_checkpoint(x, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

/content/glow-tts
/logs/pretrained.pth
INFO:root:Loaded checkpoint '/logs/pretrained.pth' (iteration 0)


In [91]:
import numpy as np

tst_stn = " Deeeep fakes are really awesome ! " # Adding spaces at the beginning and the end of utterance improves quality
sequence = np.array(text_to_sequence(tst_stn, ['english_cleaners'], cmu_dict))[None, :]
print("".join([symbols[c] for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

 deeeep @F@EY1@K@S @AA1@R @R@IH1@L@IY0 @AO1@S@AH0@M ! 


In [92]:
import IPython.display as ipd

with torch.no_grad():
  noise_scale = .667
  length_scale = 1.0
  (y_gen_tst, *r), attn_gen, *_ = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  try:
    audio = waveglow.infer(y_gen_tst.half(), sigma=.666)
  except:
    audio = waveglow.infer(y_gen_tst, sigma=.666)
ipd.Audio(audio[0].clamp(-1,1).data.cpu().float().numpy(), rate=hps.data.sampling_rate)